![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Financial Deidentification

## Colab Setup

In [1]:
import sys
import json
import os
license_key = "/home/jovyan/work/shared/gokhan/keys/4.0.2.spark_nlp_for_healthcare .json"

with open(license_key) as f:
    license_keys = json.load(f)
    
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
import sparknlp
import sparknlp_jsl

print("sparknlp version:",sparknlp.version())
print("sparknlp_jsl version:", sparknlp_jsl.version())

sparknlp version: 4.0.2
sparknlp_jsl version: 4.0.2


In [3]:
import sparknlp

#spark = sparknlp.start(gpu = True) # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)
import json
import os

import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
import os
import pandas as pd, csv, numpy as np, re
from sparknlp.training import CoNLL
import pyspark.sql.functions as F
from tqdm import tqdm
import numpy as np
from sparknlp.training import CoNLL
import re
import copy
from faker import Faker
import random


spark = sparknlp_jsl.start(license_keys['SECRET'])
print("sparknlp version:",sparknlp.version())
print("sparknlp_jsl version:", sparknlp_jsl.version())
builder = SparkSession.builder \
.appName("Spark NLP Licensed") \
.master("local[*]") \
.config("spark.driver.memory", "54G")\
.config("spark.executor.heartbeatInterval", "60s")\
.config("spark.executor.memory", "6G")\
.config("spark.driver.maxResultSize", "8G")\
.config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
.config("spark.kryoserializer.buffer.max", "2000M") \
.config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:"+ PUBLIC_VERSION) \
.config("spark.jars", "https://pypi.johnsnowlabs.com/"+ SECRET +"/spark-nlp-jsl-"+JSL_VERSION+".jar")
spark = builder.getOrCreate()

Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
:: loading settings :: url = jar:file:/home/jovyan/work/shared/venvs/gokhan/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0a60d814-0822-43a9-a3cd-36d210aa75bf;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.0.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.

sparknlp version: 4.0.2
sparknlp_jsl version: 4.0.2


# Deidentification Model

Some financial information can be considered sensitive. (e.g.,document, organization, address, signer)

In [4]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl","xx")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")
    #.setCustomBounds(["\n\n"])

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

embeddings = RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

fin_ner = MedicalNerModel.load("./medicalner_parties/outputs/1/cuad_ner") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner") 
    #.setLabelCasing("upper")

ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")\
    .setReplaceLabels({"ALIAS": "PARTY"}) # Replace "ALIAS" entity as "PARTY"

nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      embeddings,
      fin_ner,
      ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[ | ]sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
Download done! Loading the resource.
[OK!]
roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ]roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ]Download done! Loading the resource.
[ / ]

[OK!]


### Pretrained NER models extracts:
- Document
- Date
- Party (Organization Name)
- Alias

In [5]:
fin_ner.getClasses()

['O',
 'I-DOC',
 'B-EFFDATE',
 'B-ALIAS',
 'I-ALIAS',
 'B-PARTY',
 'I-EFFDATE',
 'B-DOC',
 'I-PARTY']

In [6]:
text = """Form of Indenture between the Company and Wells Fargo Bank , as trustee - incorporated by reference to Exhibit 4.3 to the Company's Registration Statement on Form S-3 , filed on August 28, 2015 """

In [7]:
result = model.transform(spark.createDataFrame([[text]]).toDF("text"))

In [8]:
result_df = result.select(F.explode(F.arrays_zip(result.token.result, 
                                                 result.ner.result)).alias("cols")) \
                  .select(F.expr("cols['0']").alias("token"),
                          F.expr("cols['1']").alias("ner_label"))

In [15]:
result_df.select("token", "ner_label").groupBy('ner_label').count().orderBy('count', ascending=False).show(truncate=False)

+---------+-----+
|ner_label|count|
+---------+-----+
|O        |27   |
|I-EFFDATE|3    |
|I-PARTY  |2    |
|B-PARTY  |1    |
|B-ALIAS  |1    |
|B-EFFDATE|1    |
+---------+-----+



### Check extracted sensitive entities

In [10]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result, 
                                     result.ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+----------------+---------+
|chunk           |ner_label|
+----------------+---------+
|Company         |PARTY    |
|Wells Fargo Bank|PARTY    |
|August 28, 2015 |EFFDATE  |
+----------------+---------+



## Masking and Obfuscation

### Replace these enitites with Tags

In [11]:
ner_converter = NerConverterInternal()\
      .setInputCols(["sentence", "token", "ner"])\
      .setOutputCol("ner_chunk") 

deidentification = DeIdentification() \
      .setInputCols(["sentence", "token", "ner_chunk"]) \
      .setOutputCol("deidentified") \
      .setMode("mask")\
      .setReturnEntityMappings(True) #  return a new column to save the mappings between the mask/obfuscated entities and original entities.
      #.setMappingsColumn("MappingCol") # change the name of the column, 'aux' is default

deidPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      embeddings,
      fin_ner,
      ner_converter,
      deidentification])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model_deid = deidPipeline.fit(empty_data)

In [12]:
result = model_deid.transform(spark.createDataFrame([[text]]).toDF("text"))

In [13]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|          embeddings|                 ner|           ner_chunk|        deidentified|                 aux|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Form of Indenture...|[[document, 0, 19...|[[document, 0, 19...|[[token, 0, 3, Fo...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 30, 36, ...|[[document, 0, 17...|[[chunk, 30, 36, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [14]:
result.select(F.explode(F.arrays_zip(result.sentence.result, result.deidentified.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("deidentified")).toPandas()

,sentence,deidentified
0,Form of Indenture between the Company and Well...,Form of Indenture between the <ALIAS> and <PAR...


We have three modes to mask the entities in the Deidentification annotator. You can select the modes using the `.setMaskingPolicy()` parameter. The methods are the followings:

**“entity_labels”**: Mask with the entity type of that chunk. (default) <br/>
**“same_length_chars”**: Mask the deid entities with same length of asterix ( * ) with brackets ( [ , ] ) on both end. <br/>
**“fixed_length_chars”**: Mask the deid entities with a fixed length of asterix ( * ). The length is setting up using the `setFixedMaskLength()` method. <br/>

Let's try each of these and compare the results:

In [16]:
#deid model with "entity_labels"
deid_entity_labels= DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk"])\
    .setOutputCol("deid_entity_label")\
    .setMode("mask")\
    .setReturnEntityMappings(True)\
    .setMaskingPolicy("entity_labels")

#deid model with "same_length_chars"
deid_same_length= DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk"])\
    .setOutputCol("deid_same_length")\
    .setMode("mask")\
    .setReturnEntityMappings(True)\
    .setMaskingPolicy("same_length_chars")

#deid model with "fixed_length_chars"
deid_fixed_length= DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk"])\
    .setOutputCol("deid_fixed_length")\
    .setMode("mask")\
    .setReturnEntityMappings(True)\
    .setMaskingPolicy("fixed_length_chars")\
    .setFixedMaskLength(4)


deidPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      embeddings,
      fin_ner,
      ner_converter,
      deid_entity_labels,
      deid_same_length,
      deid_fixed_length])


empty_data = spark.createDataFrame([[""]]).toDF("text")
model_deid = deidPipeline.fit(empty_data)

In [17]:
policy_result = model_deid.transform(spark.createDataFrame([[text]]).toDF("text"))

In [18]:
policy_result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|          embeddings|                 ner|           ner_chunk|   deid_entity_label|                 aux|    deid_same_length|   deid_fixed_length|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Form of Indenture...|[[document, 0, 19...|[[document, 0, 19...|[[token, 0, 3, Fo...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 30, 36, ...|[[document, 0, 17...|[[chunk, 30, 33, ...|[[document, 0, 19...|[[document, 0, 16...|
+--------------------+--------------------+--------------------+----

In [19]:
policy_result.select(F.explode(F.arrays_zip(policy_result.sentence.result, 
                                            policy_result.deid_entity_label.result, 
                                            policy_result.deid_same_length.result, 
                                            policy_result.deid_fixed_length.result)).alias("cols")) \
             .select(F.expr("cols['0']").alias("sentence"),
                     F.expr("cols['1']").alias("deid_entity_label"),
                     F.expr("cols['2']").alias("deid_same_length"),
                     F.expr("cols['3']").alias("deid_fixed_length")).toPandas()

,sentence,deid_entity_label,deid_same_length,deid_fixed_length
0,Form of Indenture between the Company and Well...,Form of Indenture between the <ALIAS> and <PAR...,Form of Indenture between the [*****] and [***...,"Form of Indenture between the **** and **** , ..."


### Mapping Column

In [20]:
result.select("aux").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|aux                                                                                                                                                                                                                                                                                                                                                                                

In [21]:
result.select(F.explode(F.arrays_zip(result.aux.metadata, 
                                     result.aux.result, 
                                     result.aux.begin, 
                                     result.aux.end)).alias("cols")) \
      .select(F.expr("cols['0']['originalChunk']").alias("chunk"),
              F.expr("cols['0']['beginOriginalChunk']").alias("beginChunk"),
              F.expr("cols['0']['endOriginalChunk']").alias("endChunk"),
              F.expr("cols['1']").alias("label"),
              F.expr("cols['2']").alias("beginLabel"),
              F.expr("cols['3']").alias("endLabel")).show(truncate=False)

+----------------+----------+--------+---------+----------+--------+
|chunk           |beginChunk|endChunk|label    |beginLabel|endLabel|
+----------------+----------+--------+---------+----------+--------+
|Company         |30        |36      |<ALIAS>  |30        |36      |
|Wells Fargo Bank|42        |57      |<PARTY>  |42        |48      |
|August 28, 2015 |178       |192     |<EFFDATE>|169       |177     |
+----------------+----------+--------+---------+----------+--------+



## Reidentification

We can use `ReIdentification` annotator to go back to the original sentence.

In [22]:
reIdentification = ReIdentification()\
    .setInputCols(["aux","deidentified"])\
    .setOutputCol("original")

In [23]:
reid_result = reIdentification.transform(result)

In [24]:
reid_result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|          embeddings|                 ner|           ner_chunk|        deidentified|                 aux|            original|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Form of Indenture...|[[document, 0, 19...|[[document, 0, 19...|[[token, 0, 3, Fo...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 30, 36, ...|[[document, 0, 17...|[[chunk, 30, 36, ...|[[document, 0, 19...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----

In [25]:
print(text)

reid_result.select('original.result').show(truncate=False)

Form of Indenture between the Company and Wells Fargo Bank , as trustee - incorporated by reference to Exhibit 4.3 to the Company's Registration Statement on Form S-3 , filed on August 28, 2015 
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                             |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[Form of Indenture between the Company and Wells Fargo Bank , as trustee - incorporated by reference to Exhibit 4.3 to the Company's Registration Statement on Form S-3 , filed on August 28, 2015]|
+------------

## Using multiple NER in the same pipeline

In [26]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

embeddings = RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

fin_ner = MedicalNerModel.load("./medicalner_parties/outputs/1/cuad_ner") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner") 
    #.setLabelCasing("upper")

ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")\
    .setReplaceLabels({"ALIAS": "PARTY"})

ner_signers = MedicalNerModel.load("./medicalner_signers_cur/outputs/1c/cuad_ner") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner_signers") 
    #.setLabelCasing("upper")

ner_converter_signers = NerConverter() \
    .setInputCols(["sentence", "token", "ner_signers"]) \
    .setOutputCol("ner_signer_chunk")

chunk_merge = ChunkMergeApproach()\
    .setInputCols("ner_signer_chunk", "ner_chunk")\
    .setOutputCol("deid_merged_chunk")

deidentification = DeIdentification() \
    .setInputCols(["sentence", "token", "deid_merged_chunk"]) \
    .setOutputCol("deidentified") \
    .setMode("mask")\
    .setIgnoreRegex(True)


nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      embeddings,
      fin_ner,
      ner_converter,
      ner_signers,
      ner_converter_signers,
      chunk_merge,
      deidentification])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[OK!]


In [83]:
text = """ ENTIRE AGREEMENT. Commvault Systems, Inc. , Name of each exchange on which registered
Common Stock, $0.01 par value
CVLT
The NASDAQ Stock Market
By : Sherly Johnson.   """ 

In [84]:
result = model.transform(spark.createDataFrame([[text]]).toDF("text"))

# fin_ner
result.select(F.explode(F.arrays_zip(result.ner_chunk.result, 
                                     result.ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+----------------------+---------+
|chunk                 |ner_label|
+----------------------+---------+
|ENTIRE AGREEMENT      |DOC      |
|Commvault Systems, Inc|PARTY    |
+----------------------+---------+



In [85]:
result = model.transform(spark.createDataFrame([[text]]).toDF("text"))

# ner_signers
result.select(F.explode(F.arrays_zip(result.ner_signer_chunk.result, 
                                     result.ner_signer_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+----------------------+--------------+
|chunk                 |ner_label     |
+----------------------+--------------+
|Commvault Systems, Inc|PARTY         |
|Sherly Johnson        |SIGNING_PERSON|
+----------------------+--------------+



In [86]:
result = model.transform(spark.createDataFrame([[text]]).toDF("text"))

# merged_chunk
result.select(F.explode(F.arrays_zip(result.deid_merged_chunk.result, 
                                     result.deid_merged_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+----------------------+--------------+
|chunk                 |ner_label     |
+----------------------+--------------+
|ENTIRE AGREEMENT      |DOC           |
|Commvault Systems, Inc|PARTY         |
|Sherly Johnson        |SIGNING_PERSON|
+----------------------+--------------+



In [87]:
result.select(F.explode(F.arrays_zip(result.sentence.result, 
                                     result.deidentified.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"),
              F.expr("cols['1']").alias("deidentified")).toPandas()

,sentence,deidentified
0,ENTIRE AGREEMENT.,<DOC>.
1,"Commvault Systems, Inc.",<PARTY>.
2,", Name of each exchange on which registered\nC...",", Name of each exchange on which registered\nC..."


## Obfuscation mode

In the obfuscation mode **DeIdentificationModel** will replace sensitive entities with random values of the same type. 


In [88]:
# This is the obfuscation dict for the new entities
obs_lines = """CTO#SIGNING_TITLE
Project Manager#SIGNING_TITLE
Sales Manager#SIGNING_TITLE
Business Manager#SIGNING_TITLE
Coordinator#SIGNING_TITLE
Officer#SIGNING_TITLE
Legal Agreement#DOC
Contract#DOC
Estate Document#DOC
official Document#DOC
Deed of Covenant#DOC
TURER INC#PARTY
Clark llc.#PARTY
SESA CO.#PARTY
John Snow Labs Inc#PARTY
MGT Trust Company, LLC.#PARTY
JAMES TURNER#SIGNING_PERSON
Juan Garcia#SIGNING_PERSON
Benjamin Dean#SIGNING_PERSON
Tommy Lee#SIGNING_PERSON
Dorothy Keen#SIGNING_PERSON
("AGREEMENT")#ALIAS
("TRADE COMPANY")#ALIAS
(the" Agreement")#ALIAS
("private company")#ALIAS
(the "Contract")#ALIAS
26-06-1990#EFFDATE
03/08/2025#EFFDATE
01/01/2045#EFFDATE
11/7/2016#EFFDATE
12-12-2022#EFFDATE """

with open ('obfuscate.txt', 'w') as f:
    f.write(obs_lines)

In [109]:
ner_converter_signers = NerConverter() \
    .setInputCols(["sentence", "token", "ner_signers"]) \
    .setOutputCol("ner_signer_chunk")

chunk_merge = ChunkMergeApproach()\
    .setInputCols("ner_signer_chunk", "ner_chunk")\
    .setOutputCol("deid_merged_chunk")

obfuscation = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_signer_chunk"]) \
    .setOutputCol("deidentified") \
    .setMode("obfuscate")\
    .setObfuscateDate(True)\
    .setObfuscateRefFile('obfuscate.txt')\
    .setObfuscateRefSource("both") #default: "faker"


nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      embeddings,
      fin_ner,
      ner_converter,
      ner_signers,
      ner_converter_signers,
      chunk_merge,
      obfuscation])

obfuscation_model = nlpPipeline.fit(empty_data)

In [136]:
text = """We have acquired businesses and technology in the past. For example, we acquired CNL Software Limited and RedSky . """

In [137]:
result = obfuscation_model.transform(spark.createDataFrame([[text]]).toDF("text"))

result.select(F.explode(F.arrays_zip(result.sentence.result, result.deidentified.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("deidentified")).toPandas()

,sentence,deidentified
0,We have acquired businesses and technology in ...,We have acquired businesses and technology in ...
1,"For example, we acquired CNL Software Limited ...","For example, we acquired MGT Trust Company, LL..."


## Use full pipeline in the Light model

In [138]:
light_model = LightPipeline(model)
annotated_text = light_model.annotate(text)
annotated_text['deidentified']

['We have acquired businesses and technology in the past.',
 'For example, we acquired <PARTY> and <PARTY> .']

In [139]:
obf_light_model = LightPipeline(obfuscation_model)
annotated_text = obf_light_model.annotate(text)
annotated_text['deidentified']

['We have acquired businesses and technology in the past.',
 'For example, we acquired MGT Trust Company, LLC. and John Snow Labs Inc .']

# Save the Pipeline and Use it from Your Local

In [140]:
model.write().overwrite().save('pipeline_deid')

In [141]:
from sparknlp.pretrained import PretrainedPipeline

deid_pipeline = PretrainedPipeline.from_disk("pipeline_deid")

In [142]:
data = spark.createDataFrame([[text]]).toDF("text")

In [143]:
deid_pipeline.model.stages

[DocumentAssembler_aa9603bd4664,
 SentenceDetector_9dfc07b9925a,
 REGEX_TOKENIZER_51409ee6ff79,
 ROBERTA_EMBEDDINGS_b915dff90901,
 MedicalNerModel_f714c7246b46,
 NER_CONVERTER_ea35cc068eef,
 MedicalNerModel_2b2f0f671f99,
 NerConverter_ee04fb1570a2,
 MERGE_8323f467d068,
 DE-IDENTIFICATION_0858e305ee72]

In [144]:
deid_pipeline.model.transform(data).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|          embeddings|                 ner|           ner_chunk|         ner_signers|    ner_signer_chunk|   deid_merged_chunk|        deidentified|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|We have acquired ...|[[document, 0, 11...|[[document, 0, 54...|[[token, 0, 1, We...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 81, 100,...|[[named_entity, 0...|[[chunk, 81, 100,...|[[chunk, 81, 100,...|[[document, 0, 54...|
+--------------------+--------------------+--------------------+----

# Pretrained Deidentification Pipeline

We have this pipeline can be used to deidentify financial information from texts.The financial information will be masked and obfuscated in the resulting text. The pipeline can mask and obfuscate `DOC`, `EFFDATE`, `PARTY`, `ALIAS`, `SIGNING_PERSON`, `SIGNING_TITLE`, `COUNTRY`, `CITY`, `STATE`, `STREET`, `ZIP`, `EMAIL`, `FAX`, `LOCATION-OTHER`, `DATE`,`PHONE` entities.

In [145]:
from sparknlp.pretrained import PretrainedPipeline

deid_pipeline = PretrainedPipeline.from_disk("finpipe_deid")

In [146]:
deid_pipeline.model.stages

[DocumentAssembler_57ba7ce8bff9,
 SentenceDetectorDLModel_8aaebf7e098e,
 REGEX_TOKENIZER_2f265bb3f6b5,
 ROBERTA_EMBEDDINGS_b915dff90901,
 BERT_EMBEDDINGS_29ce72cd673e,
 MedicalNerModel_f714c7246b46,
 NerConverter_5a5bb98a24c7,
 MedicalNerModel_2b2f0f671f99,
 NerConverter_8b80797c7f67,
 MedicalNerModel_7b3b98b32784,
 NER_CONVERTER_fb28b23bc35d,
 MedicalNerModel_419e708135cb,
 NER_CONVERTER_af60235365b4,
 CONTEXTUAL-PARSER_85a13a5ff4bd,
 CONTEXTUAL-PARSER_bf8f02fb6658,
 REGEX_MATCHER_6199c32417bc,
 REGEX_MATCHER_2d694c8416b8,
 MERGE_5b96d578aa9b,
 DE-IDENTIFICATION_3d3dd57f734a,
 DE-IDENTIFICATION_471d94c72cd0,
 DE-IDENTIFICATION_29cac8c6cf56,
 DE-IDENTIFICATION_407b57c7d657,
 Finisher_ed29d709e530]

In [179]:
text= """ REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF
Commvault Systems, Inc.  
(Exact name of registrant as specified in its charter) 
Signed By : Sherly Johnson
(Address of principal executive offices, including zip code) 
(732) 870-4000
(telephone number, including area code) 
Name of each exchange on which registered
CVLT
The NASDAQ Stock Market
"""

In [180]:
deid_res= deid_pipeline.annotate(text)

In [181]:
deid_res.keys()

dict_keys(['obfuscated', 'deidentified', 'masked_fixed_length_chars', 'deid_merged_chunk', 'sentence', 'masked_with_chars'])

In [182]:
pd.set_option("display.max_colwidth", 100)

df= pd.DataFrame(list(zip(deid_res["sentence"], 
                          deid_res["deidentified"],
                          deid_res["masked_with_chars"],
                          deid_res["masked_fixed_length_chars"], 
                          deid_res["obfuscated"])),
                 columns= ["Sentence", "Masked", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])

df

,Sentence,Masked,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF,REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF,REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF,REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF,REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF
1,"Commvault Systems, Inc. \n(Exact name of registrant as specified in its charter)",<PARTY> \n(Exact name of registrant as specified in its charter),[*********************] \n(Exact name of registrant as specified in its charter),**** \n(Exact name of registrant as specified in its charter),John Snow Labs Inc \n(Exact name of registrant as specified in its charter)
2,Signed By : Sherly Johnson,Signed By : <SIGNING_PERSON>,Signed By : [************],Signed By : ****,Signed By : Dorothy Keen
3,"(Address of principal executive offices, including zip code) \n(732) 870-4000\n(telephone number...","(Address of principal executive offices, including zip code) \n<PHONE>\n(telephone number, inclu...","(Address of principal executive offices, including zip code) \n[************]\n(telephone number...","(Address of principal executive offices, including zip code) \n****\n(telephone number, includin...","(Address of principal executive offices, including zip code) \n031460 3797\n(telephone number, i..."
4,Name of each exchange on which registered,Name of each exchange on which registered,Name of each exchange on which registered,Name of each exchange on which registered,Name of each exchange on which registered
5,CVLT,<PARTY>,[**],****,TURER INC
6,The NASDAQ Stock Market,The <PARTY>,The [*****************],The ****,The TURER INC
